<center><h1>Least Squares Support Vector Classifier</h1></center>

Author: Rômulo Drumond ([GitHub](https://github.com/RomuloDrumond))

## Summary:
1. [Introduction](#introduction)

2. [Using the classifier](#using_classifier)

    2.1 [CPU/Numpy version](#cpu_version)
    
    2.2 [GPU/PyTorch version](#gpu_version)

# 1. Introduction <a class="anchor" id="introduction"></a>

The Least-Squares Support Vector Machine (LSSVM) is a variation of the original Support Vector Machine (SVM) in which we have a slight change in the objective and restriction functions that results in a big simplification of the optimization problem.

First, let's see the optimization problem of an SVM:

$$ 
\begin{align}
    minimize && f_o(\vec{w},\vec{\xi})=\frac{1}{2} \vec{w}^T\vec{w} + C \sum_{i=1}^{n} \xi_i &&\\
    s.t. && y_i(\vec{w}^T\vec{x}_i+b)\geq 1 - \xi_i, && i = 1,..., n \\
         && \xi_i \geq 0,                            && i = 1,..., n
\end{align}
$$

In this case, we have a set of inequality restrictions and when solving the optimization problem by it's dual we find a discriminative function, adding the kernel trick, of the type:


$$ f(\vec{x}) = sign \ \Big( \sum_{i=1}^{n} \alpha_i^o y_i K(\vec{x}_i,\vec{x}) + b_o \Big) $$

Where $\alpha_i^o$ and $b_o$ denote optimum values. Giving enough regularization (smaller values of $C$) we get a lot of $\alpha_i^o$ nulls, resulting in a sparse model in which we only need to save the pairs $(\vec{x}_i,y_i)$ which have the optimum dual variable not null. The vectors $\vec{x}_i$ with not null $\alpha_i^o$ are known as support vectors (SV).



In the LSSVM case, we change the inequality restrictions to equality restrictions. As the $\xi_i$ may be negative we square its values in the objective function:

$$ 
\begin{align}
    minimize && f_o(\vec{w},\vec{\xi})=\frac{1}{2} \vec{w}^T\vec{w} + \gamma \frac{1}{2}\sum_{i=1}^{n} \xi_i^2 &&\\
    s.t. && y_i(\vec{w}^T\vec{x}_i+b) = 1 - \xi_i, && i = 1,..., n
\end{align}
$$


The dual of this optimization problem results in a system of linear equations, a set of Karush-Khun-Tucker (KKT) equations:

$$
\begin{bmatrix} 
    0 & \vec{d}^T \\
    \vec{y} & \Omega + \gamma^{-1} I 
\end{bmatrix}
\
\begin{bmatrix} 
    b  \\
    \vec{\alpha}
\end{bmatrix}
=
\begin{bmatrix} 
    0 \\
    \vec{1}
\end{bmatrix}
$$

Where, with the kernel trick, &nbsp; $\Omega_{i,j} = y_i y_j K(\vec{x}_i,\vec{x}_j)$,  &nbsp;  $\vec{y} = [y_1 \ y_2 \ ... \ y_n]^T$, &nbsp; $\vec{\alpha} = [\alpha_1 \ \alpha_2 \ ... \ \alpha_n]^T$ &nbsp;  e &nbsp; $\vec{1} = [1 \ 1 \ ... \ 1]^T$.

The discriminative function of the LSSVM has the same form of the SVM but the $\alpha_i^o$ aren't usually null, resulting in a bigger model. The big advantage of the LSSVM is in finding it's parameters, which is reduced to solving the linear system of the type:

$$ A\vec{z} = \vec{b} $$

A well-known solution of the linear system is when we minimize the square of the residues, that can be written as the optimization problem:

$$
\begin{align}
    minimize && f_o(\vec{z})=\frac{1}{2}||A\vec{z} - \vec{b}||^2\\
\end{align}
$$

And have the analytical solution:

$$ \vec{z} = A^{\dagger} \vec{b} $$

Where $A^{\dagger}$ is the pseudo-inverse defined as:

$$ A^{\dagger} = (A^T A)^{-1} A^T$$

# 2. Using the classifier <a class="anchor" id="using_classifier"></a>

## 2.1 CPU/Numpy version <a class="anchor" id="cpu_version"></a>

In [1]:
# Little trick to import the toolbox from the parent folder
import sys
sys.path.insert(0,'..')

from toolbox.models.svm import LSSVC

In [2]:
import numpy as np

# convert dummies to multilabel
def dummie2multilabel(X):
    N = len(X)
    X_multi = np.zeros((N,1),dtype='int')
    for i in range(N):
        temp = np.where(X[i]==1)[0] # find where 1 is found in the array
        if temp.size == 0: # is a empty array, there is no '1' in the X[i] array
            X_multi[i] = 0 # so we denote this class '0'
        else:
            X_multi[i] = temp[0] + 1
    return X_multi.T[0]

In [3]:
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits
from sklearn.preprocessing import MinMaxScaler

X, y = load_digits(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=2020)

# Scaling features
scaler = MinMaxScaler()
scaler.fit(X_train)
X_tr_norm = scaler.transform(X_train)
X_ts_norm = scaler.transform(X_test)

The user my find different params for each kernel:

In [4]:
print('Gaussian kernel:')
lssvc = LSSVC(gamma=1, kernel='rbf', sigma=.5) # Class instantiation
lssvc.fit(X_tr_norm, y_train) # Fitting the model
y_pred = lssvc.predict(X_ts_norm) # Making preditictions with trained model
print('acc_test = ', accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)), '\n')

print('Polynomial kernel:')
lssvc = LSSVC(gamma=1, kernel='poly', d=2)
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
print('acc_test = ', accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)), '\n')

print('Linear kernel:')
lssvc = LSSVC(gamma=1, kernel='linear')
lssvc.fit(X_tr_norm, y_train)
y_pred = lssvc.predict(X_ts_norm)
print('acc_test = ', accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)), '\n')

Gaussian kernel:
acc_test =  0.967741935483871 

Polynomial kernel:
acc_test =  0.9899888765294772 

Linear kernel:
acc_test =  0.978865406006674 



If in doubt, the user can run:

In [5]:
help(LSSVC.fit)

Help on function fit in module toolbox.models.svm.LSSVC:

fit(self, X, y)
    Fits the model given the set of X attribute vectors and y labels.
    - X: ndarray of shape (n_samples, n_attributes)
    - y: ndarray of shape (n_samples,) or (n_samples, n)
        If the label is represented by an array of n elements, the y 
        parameter must have n columns.



Or for a complete view:

In [6]:
help(LSSVC)

Help on class LSSVC in module toolbox.models.svm.LSSVC:

class LSSVC(builtins.object)
 |  LSSVC(gamma=1, kernel='rbf', **kernel_params)
 |  
 |  Class that implements the Least Squares Support Vector Machine for
 |  classification tasks
 |  
 |  It uses Numpy pseudo-inverse function to solve the dual optimization 
 |  problem  with ordinary least squares. In multiclass classification problems
 |  the approach used is one-vs-all, so, a model is fit for each class while
 |  considering the others a set of the same class.
 |  
 |  # Parameters:
 |  - gamma: float, default = 1.0
 |      Constant that control the regularization of the model, it may vary 
 |      in the set (0, +infinity). The closest gamma is to zero, the more 
 |      regularized the model will be.
 |  - kernel: {'linear', 'poly', 'rbf'}, default = 'rbf'
 |      The kernel name utilized for the model, if set to 'linear' the model 
 |      will not take advantage of the kernel trick, and the LSSVC maybe only
 |      useful 

The user may also save the model in JSON format:

In [7]:
lssvc.dump('model')
loaded_model = LSSVC.load('model')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc.predict(X_ts_norm))
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(loaded_model.predict(X_ts_norm))
    )
)

acc_test =  0.978865406006674
acc_test =  0.978865406006674


## 2.2 GPU/PyTorch version <a class="anchor" id="gpu_version"></a>    

In [8]:
from toolbox.models.svm import LSSVC_GPU

print('Gaussian kernel:')
lssvc_gpu = LSSVC_GPU(gamma=1, kernel='rbf', sigma=.5) 
lssvc_gpu.fit(X_tr_norm, y_train)
y_pred = lssvc_gpu.predict(X_ts_norm).cpu()
print('acc_test = ', accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)), '\n')

print('Polynomial kernel:')
lssvc_gpu = LSSVC_GPU(gamma=1, kernel='poly', d=2)
lssvc_gpu.fit(X_tr_norm, y_train)
y_pred = lssvc_gpu.predict(X_ts_norm).cpu()
print('acc_test = ', accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)), '\n')

print('Linear kernel:')
lssvc_gpu = LSSVC_GPU(gamma=1, kernel='linear')
lssvc_gpu.fit(X_tr_norm, y_train)
y_pred = lssvc_gpu.predict(X_ts_norm).cpu()
print('acc_test = ', accuracy_score(dummie2multilabel(y_test), dummie2multilabel(y_pred)), '\n')

Gaussian kernel:
acc_test =  0.967741935483871 

Polynomial kernel:
acc_test =  0.9899888765294772 

Linear kernel:
acc_test =  0.978865406006674 



In [9]:
lssvc_gpu.dump('model')
loaded_model = LSSVC_GPU.load('model')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc_gpu.predict(X_ts_norm).cpu())
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(loaded_model.predict(X_ts_norm).cpu())
    )
)

acc_test =  0.978865406006674
acc_test =  0.978865406006674


Can even change between CPU and GPU version:

In [10]:
lssvc.dump('model_from_cpu')
lssvc_gpu = LSSVC_GPU.load('model_from_cpu')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc.predict(X_ts_norm))
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc_gpu.predict(X_ts_norm).cpu())
    )
)

acc_test =  0.978865406006674
acc_test =  0.978865406006674


In [11]:
lssvc_gpu.dump('model_from_gpu')
lssvc = LSSVC.load('model_from_gpu')

# Showing the same results
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc_gpu.predict(X_ts_norm).cpu())
    )
)
print('acc_test = ', accuracy_score(
        dummie2multilabel(y_test), 
        dummie2multilabel(lssvc.predict(X_ts_norm))
    )
)

acc_test =  0.978865406006674
acc_test =  0.978865406006674
